In [1]:
%load_ext autoreload
%autoreload 2
!pwd

/home/local/USHERBROOKE/levj1404/Documents/ift714/UDS-IFT714-LanguesNaturelles


In [2]:
import argparse
import torch

from data.data_utils import load_url_dataset

In [3]:
# Load the dataset
num_workers = 2
batch_size = 64
splits_directory = "data/splits"

(trainloader, validloader, testloader), classes = load_url_dataset(splits_directory, batch_size, num_workers=num_workers)

In [4]:
import transformers

from models.roberta import RobertaUrl
from trainers.roberta_trainer import RobertaTrainer
import utils.torch_utils as ptu

nb_training_steps = 1000
max_seq_length = 512
device = ptu.get_device()

# Load the model
model = RobertaUrl().to(device)
tokenizer = transformers.RobertaTokenizer.from_pretrained("roberta-base")
loss_fn = torch.nn.BCELoss()
optimizer = torch.optim.AdamW(model.parameters(), lr=1e-5)
scheduler = transformers.get_scheduler("linear", optimizer, num_warmup_steps=0, num_training_steps=nb_training_steps)

trainer = RobertaTrainer(model,
                         tokenizer,
                         loss_fn,
                         optimizer,
                         scheduler, 
                         trainloader,
                         validloader,
                         testloader,
                         classes,
                         device=device,
                         max_seq_length=max_seq_length)

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [ ]:
trainer.train()

In [9]:
test_hist = trainer.test()

last_metrics = {'loss': test_hist['loss'][-1], 'accuracy': test_hist['accuracy'][-1]}



test:   0%|          | 0/704 [00:00<?, ?it/s]

test:   0%|          | 2/704 [00:00<01:55,  6.08it/s]

test:   0%|          | 3/704 [00:00<02:27,  4.75it/s]

test:   1%|          | 4/704 [00:00<02:43,  4.29it/s]

test:   1%|          | 5/704 [00:01<02:51,  4.08it/s]

test:   1%|          | 6/704 [00:01<02:55,  3.99it/s]

test:   1%|          | 7/704 [00:01<03:01,  3.85it/s]

test:   1%|          | 8/704 [00:01<03:00,  3.85it/s]

test:   1%|▏         | 9/704 [00:02<03:01,  3.83it/s]

test:   1%|▏         | 10/704 [00:02<03:00,  3.85it/s]

test:   2%|▏         | 11/704 [00:02<03:00,  3.83it/s]

test:   2%|▏         | 12/704 [00:02<02:59,  3.85it/s]

test:   2%|▏         | 13/704 [00:03<03:00,  3.83it/s]

test:   2%|▏         | 14/704 [00:03<03:00,  3.83it/s]

test:   2%|▏         | 15/704 [00:03<03:01,  3.80it/s]

test:   2%|▏         | 16/704 [00:04<03:00,  3.81it/s]

test:   2%|▏         | 17/704 [00:04<02:59,  3.82it/s]

test:   3%|▎         | 18/704 [00:04<02:59,  3.83it/s]

test:  

In [10]:
last_metrics

{'loss': 0.8144862326537773, 'accuracy': 0.2588113337940567}